In [11]:
import os, pathlib, datetime as dt
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import yfinance as yf

RAW = pathlib.Path('../data/raw'); RAW.mkdir(parents=True, exist_ok=True)
load_dotenv(); print('ALPHAVANTAGE_API_KEY loaded?', bool(os.getenv('ALPHAVANTAGE_API_KEY')))

ALPHAVANTAGE_API_KEY loaded? True


In [12]:
def ts():
    return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

def save_csv(df: pd.DataFrame, prefix: str, **meta):
    mid = '_'.join([f"{k}-{v}" for k,v in meta.items()])
    path = RAW / f"{prefix}_{mid}_{ts()}.csv"
    df.to_csv(path, index=False)
    print('Saved', path)
    return path

def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    return {'missing': missing, 'shape': df.shape, 'na_total': int(df.isna().sum().sum())}

In [16]:
import os, requests, pandas as pd
from dotenv import load_dotenv

load_dotenv()
ALPHA_KEY = os.getenv("ALPHAVANTAGE_API_KEY")

START_DATE = "2007-01-01"
END_DATE   = "2025-07-31"

def fetch_alpha_vig(symbol="VIG") -> pd.DataFrame:
    """Fetch daily adjusted close for VIG ETF."""
    url = "https://www.alphavantage.co/query"
    params = {
        "function": "TIME_SERIES_DAILY",
        "symbol": symbol,
        "outputsize": "full",
        "apikey": ALPHA_KEY,
    }
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    key = [k for k in js if "Time Series" in k][0]
    df = (pd.DataFrame(js[key]).T
            .reset_index()
            .rename(columns={"index": "date", "4. close": symbol}))
    df["date"] = pd.to_datetime(df["date"])
    df[symbol] = pd.to_numeric(df[symbol], errors="coerce")
    return df[["date", symbol]].query("@START_DATE <= date <= @END_DATE")

def fetch_alpha_indicator(function: str, rename: str, **kwargs) -> pd.DataFrame:
    """Fetch a monthly Alpha Vantage indicator and return date + renamed value."""
    url = "https://www.alphavantage.co/query"
    params = {"function": function, "apikey": ALPHA_KEY, **kwargs}
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    data = r.json().get("data", [])
    if not data:
        raise RuntimeError(f"No data returned for {function}")
    df = pd.DataFrame(data)
    df = df.rename(columns={"date": "date", "value": rename})
    df["date"] = pd.to_datetime(df["date"])
    df[rename] = pd.to_numeric(df[rename], errors="coerce")
    return df[["date", rename]].query("@START_DATE <= date <= @END_DATE")

def fetch_all_factors():
    """Pull VIG + CPI + Unemployment + Yield Spread into one wide DataFrame."""
    # VIG daily
    vig_df = fetch_alpha_vig("VIG")

    # Monthly macro
    cpi_df  = fetch_alpha_indicator("CPI", "CPI")
    unemp_df= fetch_alpha_indicator("UNEMPLOYMENT", "UNEMPLOYMENT")
    y10_df  = fetch_alpha_indicator("TREASURY_YIELD", "Y10", interval="monthly", maturity="10year")
    y2_df   = fetch_alpha_indicator("TREASURY_YIELD", "Y2", interval="monthly", maturity="2year")

    # Spread = 10y - 2y
    spread_df = y10_df.merge(y2_df, on="date", how="inner")
    spread_df["SPREAD"] = spread_df["Y10"] - spread_df["Y2"]
    spread_df = spread_df[["date", "SPREAD"]]

    # Merge monthly factors
    monthly = cpi_df.merge(unemp_df, on="date", how="outer")
    monthly = monthly.merge(spread_df, on="date", how="outer")

    # Enforce date range
    monthly = monthly.query("@START_DATE <= date <= @END_DATE").sort_values("date")

    return vig_df, monthly


In [17]:
vig_df, monthly_df = fetch_all_factors()

print("VIG sample (daily):")
print(vig_df.head())

print("Macro sample (monthly):")
print(monthly_df.head())


VIG sample (daily):
         date     VIG
19 2025-07-31  206.06
20 2025-07-30  207.67
21 2025-07-29  208.38
22 2025-07-28  208.67
23 2025-07-25  209.14
Macro sample (monthly):
        date      CPI  UNEMPLOYMENT  SPREAD
0 2007-01-01  202.416           4.6   -0.12
1 2007-02-01  203.499           4.5   -0.13
2 2007-03-01  205.352           4.4   -0.01
3 2007-04-01  206.686           4.5    0.02
4 2007-05-01  207.949           4.4   -0.02


In [18]:
vig_df

,date,VIG
19,2025-07-31,206.0600
20,2025-07-30,207.6700
21,2025-07-29,208.3800
22,2025-07-28,208.6700
23,2025-07-25,209.1400
...,...,...
4688,2007-01-09,53.8600
4689,2007-01-08,53.7800
4690,2007-01-05,53.6200
4691,2007-01-04,54.0096


In [22]:
vig_monthly = (
    vig_df.set_index("date")
       .resample("MS")["VIG"]
       .median()
       .rename("VIG")
       .reset_index()
)

vig_monthly.head()

,date,VIG
0,2007-01-01,54.29500
1,2007-02-01,55.10000
2,2007-03-01,53.49500
3,2007-04-01,55.25925
4,2007-05-01,56.77005


In [23]:
merged = vig_monthly.merge(monthly_df, on="date", how="left").sort_values("date").reset_index(drop=True)
merged

,date,VIG,CPI,UNEMPLOYMENT,SPREAD
0,2007-01-01,54.29500,202.416,4.6,-0.12
1,2007-02-01,55.10000,203.499,4.5,-0.13
2,2007-03-01,53.49500,205.352,4.4,-0.01
3,2007-04-01,55.25925,206.686,4.5,0.02
4,2007-05-01,56.77005,207.949,4.4,-0.02
...,...,...,...,...,...
218,2025-03-01,195.09000,319.799,4.2,0.31
219,2025-04-01,186.02000,320.795,4.2,0.50
220,2025-05-01,195.60000,321.465,4.2,0.50
221,2025-06-01,200.73500,322.561,4.1,0.49


In [5]:
SYMBOL = 'VIG'
USE_ALPHA = bool(os.getenv('ALPHAVANTAGE_API_KEY'))
if USE_ALPHA:
    url = 'https://www.alphavantage.co/query'
    params = {'function':'TIME_SERIES_DAILY','symbol':SYMBOL,'outputsize':'full','apikey':os.getenv('ALPHAVANTAGE_API_KEY')}
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
js
#     key = [k for k in js if 'Time Series' in k][0]
#     df_api = pd.DataFrame(js[key]).T.reset_index().rename(columns={'index':'date','4. close':'close'})[['date','close']]
#     df_api['date'] = pd.to_datetime(df_api['date']); df_api['close'] = pd.to_numeric(df_api['close'])
# else:
#     import yfinance as yf
#     df_api = yf.download(SYMBOL, period='3mo', interval='1d').reset_index()[['Date','Adj Close']]
#     df_api.columns = ['date','close']

# v_api = validate(df_api, ['date','close'])
# v_api


{'Information': 'We have detected your API key as JU1LZKHM3JDDYAA2 and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}

In [25]:
def fetch_alpha_indicator(function: str, **kwargs) -> pd.DataFrame:
    """
    Fetch an Alpha Vantage economic indicator and return tidy df: date, symbol, close
    """
    url = "https://www.alphavantage.co/query"
    params = {"function": function, "apikey": os.getenv("ALPHAVANTAGE_API_KEY"), **kwargs}
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    data = r.json().get("data", [])
    if not data:
        raise RuntimeError(f"No data returned for {function}")
    
    df = pd.DataFrame(data)
    df = df.rename(columns={"value": "close", "date": "date"})
    df["date"] = pd.to_datetime(df["date"])
    df["close"] = pd.to_numeric(df["close"], errors="coerce")
    df["symbol"] = function
    return df[["date","symbol","close"]].dropna()


In [26]:
# Only run if ALPHAVANTAGE_API_KEY is set
if USE_ALPHA:
    # CPI (monthly)
    cpi_df = fetch_alpha_indicator("CPI")

    # Unemployment rate (as a proxy confidence indicator)
    unemp_df = fetch_alpha_indicator("UNEMPLOYMENT")

    # Treasury Yield 10y and 2y → compute spread
    yield10 = fetch_alpha_indicator("TREASURY_YIELD", interval="monthly", maturity="10year")
    yield2  = fetch_alpha_indicator("TREASURY_YIELD", interval="monthly", maturity="2year")
    spread_df = yield10.merge(yield2, on="date", suffixes=("_10y","_2y"))
    spread_df["close"] = spread_df["close_10y"] - spread_df["close_2y"]
    spread_df = spread_df[["date"]].assign(symbol="T10Y2Y", close=spread_df["close"])

# Ensure all have columns: date, close, symbol
df_vig = df_api.copy()
df_vig["symbol"] = "VIG"

# Keep only needed columns and rename 'close'
vig_wide   = df_vig[["date","close"]].rename(columns={"close":"VIG"})
cpi_wide   = cpi_df[["date","close"]].rename(columns={"close":"CPI"})
unemp_wide = unemp_df[["date","close"]].rename(columns={"close":"UNEMPLOYMENT"})
spread_wide= spread_df[["date","close"]].rename(columns={"close":"T10Y2Y"})

# Merge step by step on 'date'
merged = vig_wide.merge(cpi_wide, on="date", how="outer")
merged = merged.merge(unemp_wide, on="date", how="outer")
merged = merged.merge(spread_wide, on="date", how="outer")

# Sort and check
merged




,date,VIG,CPI,UNEMPLOYMENT,T10Y2Y
0,1913-01-01,NaN,9.8,NaN,NaN
1,1913-02-01,NaN,9.8,NaN,NaN
2,1913-03-01,NaN,9.8,NaN,NaN
3,1913-04-01,NaN,9.8,NaN,NaN
4,1913-05-01,NaN,9.7,NaN,NaN
...,...,...,...,...,...
6061,2025-08-20,209.85,NaN,NaN,NaN
6062,2025-08-21,209.02,NaN,NaN,NaN
6063,2025-08-22,211.39,NaN,NaN,NaN
6064,2025-08-25,209.83,NaN,NaN,NaN


In [27]:
spread_df

,date,symbol,close
0,2025-07-01,T10Y2Y,0.51
1,2025-06-01,T10Y2Y,0.49
2,2025-05-01,T10Y2Y,0.50
3,2025-04-01,T10Y2Y,0.50
4,2025-03-01,T10Y2Y,0.31
...,...,...,...
585,1976-10-01,T10Y2Y,1.43
586,1976-09-01,T10Y2Y,1.17
587,1976-08-01,T10Y2Y,1.14
588,1976-07-01,T10Y2Y,0.98


In [ ]:
_ = save_csv(merged, prefix='raw', site='project', table='data')

Saved ../data/raw/scrape_site-example_table-markets_20250828-153146.csv
